🔴 Avancé | ⏱ 60 min | 🔑 Concepts : merge, pivot, apply, window functions, MultiIndex

# Pandas Avancé : Transformations et Optimisations

## Objectifs

À la fin de ce notebook, vous serez capable de :
- Fusionner plusieurs DataFrames (merge, join, concat)
- Pivoter et remodeler les données
- Appliquer des fonctions personnalisées
- Utiliser les window functions (rolling, expanding, ewm)
- Manipuler les MultiIndex
- Gérer les valeurs manquantes
- Optimiser la mémoire et les performances

## Prérequis

- Pandas bases (Series, DataFrame, groupby)
- NumPy
- Compréhension des concepts de bases de données (jointures)

In [ ]:
import pandas as pd
import numpy as np

print(f"Pandas version : {pd.__version__}")

## 1. Fil Rouge : Dataset E-commerce + Clients

Nous réutilisons le dataset de ventes et ajoutons une table clients.

In [ ]:
# Dataset des commandes (comme précédemment)
np.random.seed(42)

data_orders = {
    "order_id": range(1, 101),
    "date": pd.date_range("2024-01-01", periods=100, freq="D"),
    "product": ["Laptop", "Phone", "Tablet", "Headphones", "Monitor"] * 20,
    "category": ["Electronics", "Electronics", "Electronics", "Audio", "Electronics"] * 20,
    "quantity": [1, 2, 1, 3, 1, 2, 1, 4, 1, 2] * 10,
    "unit_price": [999.99, 699.99, 449.99, 79.99, 349.99] * 20,
    "customer_id": [f"C{i:03d}" for i in range(1, 101)],
    "city": ["Paris", "Lyon", "Marseille", "Toulouse", "Bordeaux"] * 20,
}

df_orders = pd.DataFrame(data_orders)
df_orders["total"] = df_orders["quantity"] * df_orders["unit_price"]

# Nouveau : table clients
rng = np.random.default_rng(42)
data_customers = {
    "customer_id": [f"C{i:03d}" for i in range(1, 121)],  # Plus de clients que de commandes
    "name": [f"Client {i}" for i in range(1, 121)],
    "email": [f"client{i}@example.com" for i in range(1, 121)],
    "signup_date": pd.date_range("2023-01-01", periods=120, freq="3D"),
    "segment": rng.choice(["Premium", "Standard", "Basic"], size=120, p=[0.2, 0.5, 0.3]),
    "country": ["France"] * 120,
}

df_customers = pd.DataFrame(data_customers)

print("Orders:", df_orders.shape)
print("Customers:", df_customers.shape)

## 2. Merge : Fusionner des DataFrames

Les jointures, comme en SQL.

In [ ]:
# INNER JOIN : uniquement les lignes présentes dans les deux tables
df_inner = pd.merge(df_orders, df_customers, on='customer_id', how='inner')
print(f"Inner join: {df_inner.shape}")
print(df_inner.head())

# LEFT JOIN : toutes les lignes de gauche, correspondances de droite
df_left = pd.merge(df_orders, df_customers, on='customer_id', how='left')
print(f"\nLeft join: {df_left.shape}")

# RIGHT JOIN : toutes les lignes de droite, correspondances de gauche
df_right = pd.merge(df_orders, df_customers, on='customer_id', how='right')
print(f"Right join: {df_right.shape}")
print(f"Clients sans commande: {df_right['order_id'].isna().sum()}")

# OUTER JOIN : toutes les lignes des deux tables
df_outer = pd.merge(df_orders, df_customers, on='customer_id', how='outer')
print(f"\nOuter join: {df_outer.shape}")

# Merge avec noms de colonnes différents
df_customers_renamed = df_customers.rename(columns={'customer_id': 'cust_id'})
df_merge_diff = pd.merge(
    df_orders, 
    df_customers_renamed, 
    left_on='customer_id', 
    right_on='cust_id'
)
print(f"\nMerge avec colonnes différentes: {df_merge_diff.shape}")

# Merge sur plusieurs colonnes
# (exemple fictif)
df_orders_copy = df_orders.copy()
df_orders_copy['country'] = 'France'
df_multi = pd.merge(
    df_orders_copy, 
    df_customers, 
    on=['customer_id', 'country']
)
print(f"\nMerge multi-colonnes: {df_multi.shape}")

# Suffixes pour les colonnes en conflit
df_merge_suffix = pd.merge(
    df_orders, 
    df_customers, 
    on='customer_id',
    suffixes=('_order', '_customer')
)
print(f"\nColonnes après merge:")
print(df_merge_suffix.columns.tolist())

## 3. Concat : Empiler des DataFrames

Combiner des DataFrames verticalement ou horizontalement.

In [ ]:
# Créer des sous-ensembles
df1 = df_orders.iloc[:30]
df2 = df_orders.iloc[30:60]
df3 = df_orders.iloc[60:]

# Concat vertical (empiler)
df_concat = pd.concat([df1, df2, df3], axis=0)
print(f"Concat vertical: {df_concat.shape}")
print(f"Index: {df_concat.index.tolist()[:10]}...")

# Reset index après concat
df_concat_reset = pd.concat([df1, df2, df3], axis=0, ignore_index=True)
print(f"\nAvec ignore_index: {df_concat_reset.index.tolist()[:10]}...")

# Concat horizontal (joindre côte à côte)
df_left = df_orders[['order_id', 'product', 'total']].head()
df_right = df_orders[['customer_id', 'city']].head()
df_concat_h = pd.concat([df_left, df_right], axis=1)
print(f"\nConcat horizontal:")
print(df_concat_h)

# Concat avec keys (MultiIndex)
df_with_keys = pd.concat(
    [df1, df2, df3], 
    keys=['Q1', 'Q2', 'Q3'],
    names=['quarter', 'row']
)
print(f"\nAvec keys (MultiIndex):")
print(df_with_keys.head(10))

## 4. Pivot Table : Tableaux Croisés Dynamiques

Réorganiser les données pour l'analyse.

In [ ]:
# Pivot simple : produit x ville
pivot = df_orders.pivot_table(
    values='total',
    index='product',
    columns='city',
    aggfunc='sum'
)
print("Ventes par produit et ville:")
print(pivot)

# Avec plusieurs agrégations
pivot_multi = df_orders.pivot_table(
    values='total',
    index='product',
    columns='city',
    aggfunc=['sum', 'mean', 'count']
)
print("\nPlusieurs agrégations:")
print(pivot_multi)

# Avec marges (totaux)
pivot_margins = df_orders.pivot_table(
    values='total',
    index='product',
    columns='city',
    aggfunc='sum',
    margins=True,
    margins_name='TOTAL'
)
print("\nAvec marges:")
print(pivot_margins)

# Pivot avec dates (agrégation temporelle)
df_orders['month'] = df_orders['date'].dt.to_period('M')
pivot_time = df_orders.pivot_table(
    values='total',
    index='month',
    columns='product',
    aggfunc='sum',
    fill_value=0
)
print("\nVentes mensuelles par produit:")
print(pivot_time.head())

## 5. Melt : Inverse du Pivot (Wide → Long)

Transformer un format large en format long.

In [ ]:
# Créer un DataFrame large
df_wide = df_orders.pivot_table(
    values='total',
    index='order_id',
    columns='product',
    aggfunc='sum'
).reset_index().head(10)

print("Format wide:")
print(df_wide)

# Melt : wide → long
df_long = df_wide.melt(
    id_vars='order_id',
    var_name='product',
    value_name='total'
)
print("\nFormat long (après melt):")
print(df_long.head(15))

# Supprimer les valeurs NaN
df_long_clean = df_long.dropna()
print(f"\nAprès nettoyage: {len(df_long)} → {len(df_long_clean)} lignes")

## 6. Apply : Appliquer des Fonctions

Appliquer une fonction à chaque élément, ligne ou colonne.

In [ ]:
# apply() sur une Series
def categorize_price(price):
    if price < 100:
        return 'Low'
    elif price < 500:
        return 'Medium'
    else:
        return 'High'

df_orders['price_category'] = df_orders['unit_price'].apply(categorize_price)
print("Catégories de prix:")
print(df_orders[['product', 'unit_price', 'price_category']].head())

# apply() avec lambda
df_orders['total_rounded'] = df_orders['total'].apply(lambda x: round(x, 0))
print("\nTotaux arrondis:")
print(df_orders[['total', 'total_rounded']].head())

# apply() sur DataFrame (par ligne)
def calculate_discount(row):
    if row['total'] > 1000:
        return row['total'] * 0.1
    return 0

df_orders['discount'] = df_orders.apply(calculate_discount, axis=1)
print("\nRemises calculées:")
print(df_orders[df_orders['discount'] > 0][['order_id', 'total', 'discount']].head())

# apply() sur colonnes (axis=0)
numeric_cols = ['quantity', 'unit_price', 'total']
stats = df_orders[numeric_cols].apply(np.mean)
print("\nMoyennes par colonne:")
print(stats)

# ATTENTION : apply() peut être lent
# Préférer les opérations vectorisées quand possible
print("\n⚠️ Performance:")
print("apply() est pratique mais peut être lent sur de gros DataFrames")
print("Préférer les opérations vectorisées (np.where, masques booléens, etc.)")

## 7. Map et Replace

Remplacer des valeurs de manière efficace.

In [ ]:
# map() : remplacer selon un dictionnaire
city_mapping = {
    'Paris': 'IDF',
    'Lyon': 'ARA',
    'Marseille': 'PACA',
    'Toulouse': 'OCC',
    'Bordeaux': 'NAQ'
}

df_orders['region'] = df_orders['city'].map(city_mapping)
print("Mapping ville → région:")
print(df_orders[['city', 'region']].head())

# map() avec fonction
df_orders['product_upper'] = df_orders['product'].map(str.upper)
print("\nProduits en majuscules:")
print(df_orders[['product', 'product_upper']].head())

# replace() : remplacement simple
df_test = df_orders.copy()
df_test['category'] = df_test['category'].replace('Electronics', 'Tech')
print("\nAprès replace:")
print(df_test['category'].value_counts())

# replace() avec dictionnaire
df_test['category'] = df_test['category'].replace({
    'Tech': 'Technology',
    'Audio': 'Sound'
})
print("\nAprès replace multiple:")
print(df_test['category'].value_counts())

## 8. Window Functions : Rolling, Expanding, EWM

Calculs sur des fenêtres glissantes.

In [ ]:
# Préparer les données : ventes quotidiennes
daily_sales = df_orders.groupby('date')['total'].sum().reset_index()
daily_sales = daily_sales.sort_values('date')

print("Ventes quotidiennes:")
print(daily_sales.head(10))

# Rolling : moyenne mobile sur 7 jours
daily_sales['rolling_7d'] = daily_sales['total'].rolling(window=7).mean()
print("\nAvec moyenne mobile 7j:")
print(daily_sales.head(10))

# Rolling avec min_periods
daily_sales['rolling_7d_min3'] = daily_sales['total'].rolling(
    window=7, 
    min_periods=3
).mean()
print("\nAvec min_periods=3:")
print(daily_sales.head(10))

# Autres agrégations rolling
daily_sales['rolling_max'] = daily_sales['total'].rolling(window=7).max()
daily_sales['rolling_std'] = daily_sales['total'].rolling(window=7).std()

print("\nStatistiques rolling:")
print(daily_sales[['date', 'total', 'rolling_7d', 'rolling_max', 'rolling_std']].tail(10))

# Expanding : moyenne cumulative
daily_sales['expanding_mean'] = daily_sales['total'].expanding().mean()
print("\nMoyenne cumulative:")
print(daily_sales[['date', 'total', 'expanding_mean']].head(10))

# EWM : moyenne mobile exponentielle
daily_sales['ewm'] = daily_sales['total'].ewm(span=7).mean()
print("\nMoyenne mobile exponentielle:")
print(daily_sales[['date', 'total', 'rolling_7d', 'ewm']].tail(10))

## 9. MultiIndex : Index Hiérarchiques

Organiser les données avec plusieurs niveaux d'index.

In [ ]:
# Créer un MultiIndex avec groupby
df_multi = df_orders.groupby(['city', 'product'])['total'].agg(['sum', 'mean', 'count'])
print("DataFrame avec MultiIndex:")
print(df_multi)
print(f"\nType d'index: {type(df_multi.index)}")
print(f"Niveaux: {df_multi.index.names}")

# Accès avec MultiIndex
print("\nVentes à Paris:")
print(df_multi.loc['Paris'])

print("\nLaptops à Paris:")
print(df_multi.loc[('Paris', 'Laptop')])

# Sélection avec slice
print("\nParis et Lyon:")
print(df_multi.loc[['Paris', 'Lyon']])

# reset_index : transformer l'index en colonnes
df_flat = df_multi.reset_index()
print("\nAprès reset_index:")
print(df_flat.head())

# set_index : créer un MultiIndex
df_multi_again = df_flat.set_index(['city', 'product'])
print("\nRecréé avec set_index:")
print(df_multi_again.head())

# Tri d'un MultiIndex
df_multi_sorted = df_multi.sort_index(level=[0, 1])
print("\nTrié:")
print(df_multi_sorted.head(10))

# Opérations sur niveaux
print("\nSomme par ville (level=0):")
print(df_multi.sum(level=0))

print("\nSomme par produit (level=1):")
print(df_multi.sum(level=1))

## 10. Gestion des Valeurs Manquantes

Détecter, supprimer et imputer les valeurs manquantes.

In [ ]:
# Créer des valeurs manquantes
df_na = df_orders.copy()
df_na.loc[5:10, 'quantity'] = np.nan
df_na.loc[15:20, 'city'] = np.nan
df_na.loc[25, 'total'] = np.nan

# Détecter les valeurs manquantes
print("Valeurs manquantes par colonne:")
print(df_na.isna().sum())

print("\nPourcentage de valeurs manquantes:")
print((df_na.isna().sum() / len(df_na) * 100).round(2))

# Lignes avec au moins une valeur manquante
print(f"\nLignes avec NaN: {df_na.isna().any(axis=1).sum()}")

# Lignes complètes (sans NaN)
print(f"Lignes complètes: {df_na.notna().all(axis=1).sum()}")

# dropna() : supprimer les lignes/colonnes avec NaN
df_dropped_rows = df_na.dropna()
print(f"\nAprès dropna(): {len(df_na)} → {len(df_dropped_rows)} lignes")

df_dropped_cols = df_na.dropna(axis=1)
print(f"Colonnes conservées: {df_dropped_cols.columns.tolist()}")

# dropna() avec seuil
df_dropped_thresh = df_na.dropna(thresh=8)  # au moins 8 valeurs non-NaN
print(f"\nAvec thresh=8: {len(df_na)} → {len(df_dropped_thresh)} lignes")

# fillna() : imputer les valeurs manquantes
df_filled = df_na.copy()
df_filled['quantity'] = df_filled['quantity'].fillna(0)
df_filled['city'] = df_filled['city'].fillna('Unknown')
print("\nAprès fillna:")
print(df_filled.isna().sum())

# Imputation par la moyenne/médiane
df_filled['total'] = df_filled['total'].fillna(df_filled['total'].median())
print("\nAprès imputation par la médiane:")
print(df_filled.isna().sum())

# Forward fill et backward fill
df_ff = df_na.copy()
df_ff['city'] = df_ff['city'].fillna(method='ffill')  # propager en avant
print("\nForward fill:")
print(df_ff.loc[13:23, ['order_id', 'city']])

df_bf = df_na.copy()
df_bf['city'] = df_bf['city'].fillna(method='bfill')  # propager en arrière
print("\nBackward fill:")
print(df_bf.loc[13:23, ['order_id', 'city']])

## 11. Gestion des Dates

Manipulation puissante des dates avec Pandas.

In [ ]:
# Conversion en datetime
df_dates = pd.DataFrame({
    'date_str': ['2024-01-01', '2024-02-15', '2024-03-30'],
    'value': [100, 200, 300]
})

df_dates['date'] = pd.to_datetime(df_dates['date_str'])
print("Conversion en datetime:")
print(df_dates.dtypes)

# dt accessor : extraire des composants
df_orders['year'] = df_orders['date'].dt.year
df_orders['month'] = df_orders['date'].dt.month
df_orders['day'] = df_orders['date'].dt.day
df_orders['dayofweek'] = df_orders['date'].dt.dayofweek  # 0=lundi
df_orders['dayname'] = df_orders['date'].dt.day_name()
df_orders['quarter'] = df_orders['date'].dt.quarter

print("\nComposants de date:")
print(df_orders[['date', 'year', 'month', 'day', 'dayname', 'quarter']].head())

# Calculs avec dates
df_orders['days_since_start'] = (df_orders['date'] - df_orders['date'].min()).dt.days
print("\nJours depuis le début:")
print(df_orders[['date', 'days_since_start']].head())

# Resample : agrégation temporelle
df_resampled = df_orders.set_index('date').resample('W')['total'].sum()
print("\nVentes hebdomadaires:")
print(df_resampled.head())

# Resample avec plusieurs agrégations
df_resampled_multi = df_orders.set_index('date').resample('M').agg({
    'total': 'sum',
    'quantity': 'sum',
    'order_id': 'count'
})
print("\nVentes mensuelles:")
print(df_resampled_multi)

## 12. Optimisation Mémoire

Réduire l'empreinte mémoire des DataFrames.

In [ ]:
# Afficher l'usage mémoire
print("Usage mémoire par colonne:")
print(df_orders.memory_usage(deep=True))
print(f"\nTotal: {df_orders.memory_usage(deep=True).sum() / 1024:.2f} KB")

# Optimiser les types numériques
df_optimized = df_orders.copy()

# int64 → int32 (si les valeurs le permettent)
df_optimized['order_id'] = df_optimized['order_id'].astype('int32')
df_optimized['quantity'] = df_optimized['quantity'].astype('int8')

# float64 → float32
df_optimized['unit_price'] = df_optimized['unit_price'].astype('float32')
df_optimized['total'] = df_optimized['total'].astype('float32')

print("\nAprès optimisation numérique:")
print(df_optimized.memory_usage(deep=True))
print(f"Total: {df_optimized.memory_usage(deep=True).sum() / 1024:.2f} KB")

# category dtype pour les colonnes répétitives
df_optimized['product'] = df_optimized['product'].astype('category')
df_optimized['category'] = df_optimized['category'].astype('category')
df_optimized['city'] = df_optimized['city'].astype('category')

print("\nAprès conversion en category:")
print(df_optimized.memory_usage(deep=True))
print(f"Total: {df_optimized.memory_usage(deep=True).sum() / 1024:.2f} KB")

# Comparaison
before = df_orders.memory_usage(deep=True).sum() / 1024
after = df_optimized.memory_usage(deep=True).sum() / 1024
print(f"\nRéduction: {before:.2f} KB → {after:.2f} KB ({(1 - after/before)*100:.1f}%)")

## 13. Pièges Courants

### Piège 1 : Apply() Lent

In [ ]:
import time

# Créer un DataFrame de test
df_test = pd.DataFrame({
    'value': range(100000)
})

# ❌ Avec apply()
start = time.time()
df_test['squared_apply'] = df_test['value'].apply(lambda x: x ** 2)
time_apply = time.time() - start

# ✅ Avec opération vectorisée
start = time.time()
df_test['squared_vectorized'] = df_test['value'] ** 2
time_vectorized = time.time() - start

print(f"apply(): {time_apply:.4f}s")
print(f"vectorized: {time_vectorized:.4f}s")
print(f"Speedup: {time_apply/time_vectorized:.1f}x")
print("\n✅ Préférer les opérations vectorisées !")

### Piège 2 : Oublier copy()

In [ ]:
# ❌ Sans copy()
df_original = pd.DataFrame({'A': [1, 2, 3]})
df_reference = df_original  # référence, pas copie
df_reference.loc[0, 'A'] = 999

print("❌ Sans copy():")
print("Original:", df_original['A'].tolist())
print("L'original a été modifié !\n")

# ✅ Avec copy()
df_original = pd.DataFrame({'A': [1, 2, 3]})
df_copy = df_original.copy()
df_copy.loc[0, 'A'] = 999

print("✅ Avec copy():")
print("Original:", df_original['A'].tolist())
print("Copie:", df_copy['A'].tolist())
print("L'original est intact !")

### Piège 3 : inplace Deprecated

In [ ]:
# ❌ inplace=True (deprecated dans de nombreuses fonctions)
# df.dropna(inplace=True)  # À éviter

# ✅ Réassignation
df_clean = df_na.dropna()
# ou
df_na = df_na.dropna()

print("✅ Utiliser la réassignation au lieu de inplace")
print("Plus clair et compatible avec le chaînage de méthodes")

## 14. Mini-Exercices

### Exercice 1 : Merge avec Table Clients

1. Faites un left join entre `df_orders` et `df_customers`
2. Calculez le nombre de commandes par segment de client (Premium, Standard, Basic)
3. Calculez le chiffre d'affaires moyen par segment
4. Identifiez les clients qui n'ont passé aucune commande

In [ ]:
# Votre code ici


### Exercice 2 : Pivot des Ventes par Mois/Produit

1. Créez un pivot table montrant les ventes totales par mois (lignes) et produit (colonnes)
2. Ajoutez les totaux (marges)
3. Identifiez le mois avec les meilleures ventes pour chaque produit
4. Calculez la croissance mensuelle pour chaque produit

In [ ]:
# Votre code ici


### Exercice 3 : Rolling Average

1. Calculez les ventes quotidiennes totales
2. Ajoutez une moyenne mobile sur 7 jours
3. Ajoutez une moyenne mobile sur 30 jours
4. Identifiez les jours où les ventes sont supérieures à la moyenne mobile 7j
5. Calculez l'écart-type mobile sur 7 jours

In [ ]:
# Votre code ici


---

## Solutions des Exercices

### Solution Exercice 1

In [ ]:
# 1. Left join
df_merged = pd.merge(df_orders, df_customers, on='customer_id', how='left')
print("1. Après merge:", df_merged.shape)

# 2. Commandes par segment
commandes_par_segment = df_merged.groupby('segment')['order_id'].count()
print("\n2. Commandes par segment:")
print(commandes_par_segment)

# 3. CA moyen par segment
ca_par_segment = df_merged.groupby('segment')['total'].mean()
print("\n3. CA moyen par segment:")
print(ca_par_segment)

# 4. Clients sans commande
df_all_customers = pd.merge(df_customers, df_orders, on='customer_id', how='left')
clients_sans_commande = df_all_customers[df_all_customers['order_id'].isna()]
print(f"\n4. Clients sans commande: {clients_sans_commande['customer_id'].nunique()}")
print(clients_sans_commande[['customer_id', 'name', 'segment']].head())

### Solution Exercice 2

In [ ]:
# Préparer les données avec mois
df_with_month = df_orders.copy()
df_with_month['month'] = df_with_month['date'].dt.to_period('M')

# 1. Pivot table
pivot_monthly = df_with_month.pivot_table(
    values='total',
    index='month',
    columns='product',
    aggfunc='sum',
    margins=True,  # 2. Avec marges
    margins_name='TOTAL'
)
print("1-2. Pivot avec marges:")
print(pivot_monthly)

# 3. Meilleur mois par produit
print("\n3. Meilleur mois par produit:")
for product in pivot_monthly.columns[:-1]:  # Exclure TOTAL
    best_month = pivot_monthly[product].idxmax()
    best_value = pivot_monthly[product].max()
    print(f"{product}: {best_month} ({best_value:.2f}€)")

# 4. Croissance mensuelle
pivot_growth = pivot_monthly.drop('TOTAL').pct_change() * 100
print("\n4. Croissance mensuelle (%)")
print(pivot_growth)

### Solution Exercice 3

In [ ]:
# 1. Ventes quotidiennes
daily = df_orders.groupby('date')['total'].sum().reset_index()
daily = daily.sort_values('date')

# 2-3. Moyennes mobiles
daily['ma_7d'] = daily['total'].rolling(window=7, min_periods=1).mean()
daily['ma_30d'] = daily['total'].rolling(window=30, min_periods=1).mean()

print("1-3. Ventes avec moyennes mobiles:")
print(daily.tail(10))

# 4. Jours au-dessus de MA 7j
daily['above_ma'] = daily['total'] > daily['ma_7d']
nb_above = daily['above_ma'].sum()
print(f"\n4. Jours au-dessus de MA 7j: {nb_above} ({nb_above/len(daily)*100:.1f}%)")

# 5. Écart-type mobile
daily['std_7d'] = daily['total'].rolling(window=7, min_periods=1).std()

print("\n5. Avec écart-type mobile:")
print(daily[['date', 'total', 'ma_7d', 'std_7d']].tail(10))